In [2]:
import sys
!{sys.executable} -m pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 18.0 MB/s eta 0:00:00a 0:00:01


In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

24/10/23 20:43:22 WARN Utils: Your hostname, APTINWLPF3VZG55 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/23 20:43:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/23 20:43:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/23 20:43:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
data = [(0, 1.0, 0.5), (1, 2.0, 1.5), (0, 0.5, 0.3), (1, 2.5, 1.7)]
df = spark.createDataFrame(data, ["label", "feature1", "feature2"])

24/10/23 20:43:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")

In [8]:
lr = LogisticRegression(featuresCol="features", labelCol="label")

In [9]:
pipeline = Pipeline(stages=[assembler, lr])

In [10]:
model = pipeline.fit(df)

24/10/23 20:46:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/23 20:46:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [12]:
predictions = model.transform(df)

In [13]:
predictions.select("label", "features", "prediction").show()

+-----+---------+----------+
|label| features|prediction|
+-----+---------+----------+
|    0|[1.0,0.5]|       0.0|
|    1|[2.0,1.5]|       1.0|
|    0|[0.5,0.3]|       0.0|
|    1|[2.5,1.7]|       1.0|
+-----+---------+----------+

